# Climate Data from Home System

In [43]:
from erddapy import ERDDAP
import pandas as pd
import datetime

# for secondary/derived parameters
from metpy.units import units
import metpy.calc as mpcalc

In [44]:
server_url = 'http://raspberrypi.local:8080/erddap'

In [45]:
e = ERDDAP(server=server_url)
df = pd.read_csv(e.get_search_url(response='csv', search_for='MoonFlower'))
print(df['Dataset ID'].values)

['tempest_moonflower_wx' 'channel_1027974_thingspeak'
 'channel_1037066_thingspeak' 'channel_1047747_thingspeak'
 'channel_843357_thingspeak' 'allDatasets']


In [46]:
dataset_id='tempest_moonflower_wx'

try:
    d = ERDDAP(server=server_url,
        protocol='tabledap',
        response='csv'
    )
    d.dataset_id=dataset_id
except HTTPError:
    print('Failed to generate url {}'.format(dataset_id))

try:
    df_m = d.to_pandas(
                index_col='time (UTC)',
                parse_dates=True,
                skiprows=(1,)  # units information can be dropped.
                )
    df_m.sort_index(inplace=True)
    df_m.columns = [x[1].split()[0] for x in enumerate(df_m.columns)]

except:
    print(f"something failed in data download {dataset_id}")
    pass

df_m.drop(columns=['device_id', 'bucket_step_minutes', 'wind_lull','wind_interval'],inplace=True)
#stats are all utc driven - but we really want local daily values
df_m=df_m.tz_convert('US/Pacific')

# calculations of various parameters... metpy?
# HDD/CDD, dewpointTemp

In [47]:
df_m['dewpointTemp']=mpcalc.dewpoint_from_relative_humidity(df_m.temperature.values * units.degC,
                                                            df_m.humidity.values * units.percent)
#wetbulb from metpy had issues

In [48]:
df_m['SLP']=df_m.statioinpressure.values * (1+((1013.25/df_m.statioinpressure.values)**((287.05*0.0065)/9.80665)) * (0.0065*87.3)/288.15)**(9.80665/(287.05*0.0065))

In [61]:
df_m.solar_radiation

time (UTC)
2020-06-25 21:04:00-07:00      7.0
2020-06-25 21:05:00-07:00      6.0
2020-06-25 21:06:00-07:00      6.0
2020-06-25 21:07:00-07:00      6.0
2020-06-25 21:08:00-07:00      5.0
                             ...  
2020-07-27 08:55:00-07:00    521.0
2020-07-27 08:56:00-07:00    526.0
2020-07-27 08:57:00-07:00    531.0
2020-07-27 08:58:00-07:00    536.0
2020-07-27 08:59:00-07:00    541.0
Name: solar_radiation, Length: 39275, dtype: float64

In [50]:
df_daily_max = df_m.resample('D').max()
df_daily_min = df_m.resample('D').min()
df_daily_ave = df_m.resample('D').mean()
df_daily_total = df_m.resample('1T').mean().resample('D').sum()
df_m.sample()

,type,wind_avg,wind_gust,wind_dir,statioinpressure,temperature,humidity,lux,uv,solar_radiation,...,strike_distance,strike_count,battery,report_interval,local_daily_precip,precip_final,local_daily_precip_final,precip_analysis_type,dewpointTemp,SLP
time (UTC),,,,,,,,,,,,,,,,,,,,,
2020-07-12 23:38:00-07:00,obs_st,0.15,0.63,127.0,1012.6,11.7,86.0,0,0.0,0.0,...,0.0,0.0,2.63,1.0,1.617252,0.0,0.0,2.0,9.437309,1023.126159


In [51]:
use_current_month = True

if use_current_month:
    current_month = datetime.datetime.now().month
else:
    current_month = 6

current_month_grid_data=pd.DataFrame()
current_month_grid_data = df_daily_max[df_daily_max.index.month==current_month].temperature
current_month_grid_data = pd.concat([current_month_grid_data,
                                     df_daily_min[df_daily_min.index.month==current_month].temperature,
                                     df_daily_ave[df_daily_ave.index.month==current_month].temperature.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].SLP.round(1),
                                     df_daily_total[df_daily_total.index.month==current_month].solar_radiation,
                                     df_daily_max[df_daily_max.index.month==current_month].uv.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_avg.round(1),
                                     df_daily_ave[df_daily_ave.index.month==current_month].wind_dir.astype(int),
                                     df_daily_max[df_daily_max.index.month==current_month].wind_gust.round(1)
                                    ],axis=1)
current_month_grid_data.columns=('max_temperature','min_temperature','mean_temperature','mean SLP','total_solar_radiation','max_uv_index','average speed','average dir','max gust')

In [52]:
#this should go to erddap
current_month_grid_data.to_csv(f'MoonflowerTempest_2020{str(current_month).zfill(2)}.csv')
current_month_grid_data

,max_temperature,min_temperature,mean_temperature,mean SLP,total_solar_radiation,max_uv_index,average speed,average dir,max gust
time (UTC),,,,,,,,,
2020-07-01 00:00:00-07:00,18.8,10.9,13.0,1015.6,208604.0,6.6,0.6,149,5.0
2020-07-02 00:00:00-07:00,15.2,9.2,12.2,1017.3,102809.0,2.3,0.5,124,4.8
2020-07-03 00:00:00-07:00,18.0,9.1,13.2,1020.6,192029.0,4.3,0.3,88,4.5
2020-07-04 00:00:00-07:00,22.7,10.3,14.6,1024.3,307947.0,8.7,0.4,74,4.3
2020-07-05 00:00:00-07:00,24.4,8.3,15.0,1023.0,508393.0,6.0,0.5,72,6.5
2020-07-06 00:00:00-07:00,22.3,8.5,14.7,1019.5,435852.0,7.1,0.5,93,5.4
2020-07-07 00:00:00-07:00,16.8,11.1,13.4,1020.4,159360.0,3.5,0.3,94,3.8
2020-07-08 00:00:00-07:00,22.7,11.0,15.6,1020.7,379207.0,6.8,0.4,77,4.7
2020-07-09 00:00:00-07:00,17.8,11.9,14.1,1018.6,169914.0,3.3,0.3,70,4.2


In [53]:
#monthly stats
current_month_grid_data.describe()

,max_temperature,min_temperature,mean_temperature,mean SLP,total_solar_radiation,max_uv_index,average speed,average dir,max gust
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,21.311111,11.118519,15.507407,1018.492593,256643.333333,5.355556,0.400000,83.185185,5.344444
std,3.332051,2.018575,1.888411,3.522338,185266.583175,2.108104,0.114354,21.908967,1.492374
min,15.200000,8.100000,12.200000,1010.200000,11403.000000,1.800000,0.100000,38.000000,2.600000
25%,18.550000,9.550000,14.550000,1015.900000,63828.000000,3.400000,0.300000,71.500000,4.300000
50%,22.100000,11.000000,15.000000,1019.400000,214883.000000,5.900000,0.400000,77.000000,5.000000
75%,24.050000,12.350000,16.650000,1020.850000,449268.000000,7.000000,0.450000,93.500000,6.400000
max,28.200000,15.900000,20.100000,1024.300000,508393.000000,8.700000,0.700000,149.000000,8.600000


In [54]:
### need to manage daily records, monthly records, alltime records